In [1]:
import random, gc, os, pickle, csv, time

import datasets.utils
import models.utils
from models.cls_oml_ori_v2 import OML
from models.base_models_ori import LabelAwareReplayMemory

import numpy as np

import higher
import torch
import torch.nn.functional as F
from torch.utils import data

# Constants

In [2]:
dataset_order_mapping = {
    1: [2, 0, 3, 1, 4],
    2: [3, 4, 0, 1, 2],
    3: [2, 4, 1, 3, 0],
    4: [0, 2, 1, 4, 3]
}
n_classes = 33
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# model_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518612.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456804.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477968.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869765.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241546.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424113.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167637.pt"

# v. SR 
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695669.pt"
# v. SR Query
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854228.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428520.pt"
# model_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904898.pt"
# model_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774382.pt"
# model_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782801.pt"
model_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061.pt"


# memory_path = "/data/model_runs/original_oml/aOML-order1-2022-07-18/OML-order1-id4-2022-07-18_17-53-13.518639_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr002-2022-07-31/OML-order1-id4-2022-07-31_14-53-46.456828_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-2022-07-31/OML-order1-id4-2022-07-31_18-47-41.477992_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr005-up20-2022-08-01/OML-order1-id4-2022-08-01_14-45-55.869797_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-07-31/OML-order1-id4-2022-07-31_21-18-36.241572_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr020-2022-08-16/OML-order1-id4-2022-08-16_11-37-19.424139_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr050-2022-08-16/OML-order1-id4-2022-08-16_14-16-12.167666_memory.pickle"
# v. SR 
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-29-sr/OML-order1-id4-2022-08-29_18-10-31.695692_memory.pickle"
# v. SR Query
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-08-30-sr-query/OML-order1-id4-2022-08-30_05-21-18.854254_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2022-14-21-sreversed/OML-order1-id4-2022-12-15_03-55-41.428551_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order1-inlr010-2023-01-03-srev-replay/OML-order1-id4-2023-01-03_02-59-30.904925_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order2-2023-01-07-srev-replay/OML-order2-id4-2023-01-07_19-01-27.774413_memory.pickle"
# memory_path = "/data/model_runs/original_oml/aOML-order3-2023-01-08-srev-replay/OML-order3-id4-2023-01-08_12-03-38.782827_memory.pickle"
memory_path = "/data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303088_memory.pickle"


# new_memory_path, ext = os.path.splitext(memory_path)
# new_memory_path = new_memory_path + "_label" + ext

use_db_cache = True
cache_dir = 'tmp'

In [3]:
args = {
    "order": 1,
    "n_epochs": 1,
    "lr": 3e-5,
    "inner_lr": 0.001*10,
    "meta_lr": 3e-5,
    "model": "bert",
    "learner": "oml",
    "mini_batch_size": 16,
    "updates": 5*1,
    "write_prob": 1.0,
    "max_length": 448,
    "seed": 42,
    "replay_rate": 0.01,
    "replay_every": 9600
}
updates = args["updates"]
mini_batch_size = args["mini_batch_size"]
order = args["order"]

In [4]:
torch.manual_seed(args["seed"])
random.seed(args["seed"])
np.random.seed(args["seed"])

# Load Dataset

In [5]:
print('Loading the datasets')
test_datasets = []
for dataset_id in dataset_order_mapping[order]:
    test_dataset_file = os.path.join(cache_dir, f"{dataset_id}.cache")
    if os.path.exists(test_dataset_file):
        with open(test_dataset_file, 'rb') as f:
            test_dataset = pickle.load(f)
    else:
        test_dataset = datasets.utils.get_dataset_test("", dataset_id)
        print('Loaded {}'.format(test_dataset.__class__.__name__))
        test_dataset = datasets.utils.offset_labels(test_dataset)
        pickle.dump(test_dataset, open( test_dataset_file, "wb" ), protocol=pickle.HIGHEST_PROTOCOL)
        print(f"Pickle saved at {test_dataset_file}")
    test_datasets.append(test_dataset)
print('Finished loading all the datasets')

Loading the datasets
Finished loading all the datasets


# Load Model

In [6]:
learner = OML(device=device, n_classes=n_classes, **args)
print('Using {} as learner'.format(learner.__class__.__name__))
learner.load_model(model_path)
with open(memory_path, 'rb') as f:
#     learner.memory = pickle.load(f)
    memory_buffer = pickle.load(f)


2023-01-12 11:18:20,437 - transformers.tokenization_utils_base - INFO - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2023-01-12 11:18:21,670 - transformers.configuration_utils - INFO - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2023-01-12 11:18:21,674 - transformers.configuration_utils - INFO - Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer

Using OML as learner


In [7]:
# Setting up task dict for task-aware
memory_buffer.task_dict = {
    0: list(range(5, 9)), # AG
    1: list(range(0, 5)), # Amazon
    2: list(range(0, 5)), # Yelp
    3: list(range(9, 23)), # DBPedia
    4: list(range(23, 33)), # Yahoo
}

In [8]:
# label to task converter - hack, since normally we would just use the task token/identifier
def get_task_from_label(label_idx, task_dict):
    for task_idx, class_list in task_dict.items():
        if label_idx in class_list: return task_idx
    return -1
print(get_task_from_label(8, memory_buffer.task_dict))
# Find mode: https://stackoverflow.com/questions/10797819/finding-the-mode-of-a-list
def mode(array):
    return max(array, key = array.count)
# label list to task id
def get_task_from_label_list(label_list, task_dict):
    return mode([get_task_from_label(label, task_dict) for label in label_list])
print(get_task_from_label_list([1,2,32,1,4,2,0], memory_buffer.task_dict))

0
1


In [9]:
dataclass_mapper = {
    "AGNewsDataset": 0,
    "AmazonDataset": 1,
    "YelpDataset": 2,
    "DBPediaDataset": 3,
    "YahooAnswersDataset": 4
}
dataclass_mapper["AGNewsDataset"]

0

# Testing

Select specific column index per row
https://stackoverflow.com/questions/23435782/numpy-selecting-specific-column-index-per-row-by-using-a-list-of-indexes

In [10]:
def evaluate(dataloader, updates, mini_batch_size, dataname=""):
    learner.rln.eval()
    learner.pln.train()
    
    all_losses, all_predictions, all_labels, all_label_conf = [], [], [], []
    all_adaptation_time = []
    # Get Query set first. and then find supporting support set
    for query_idx, (query_text, query_labels) in enumerate(dataloader):
        print(f"Query ID {query_idx}/{len(dataloader)}")
        # The task id to optimize to for support set
        # task_idx = get_task_from_label_list(query_labels, memory_buffer.task_dict)
        task_idx = dataclass_mapper[dataname]
        
    
        support_set = []
        for _ in range(updates):
            text, labels = memory_buffer.read_batch_task(batch_size=mini_batch_size, task_idx=task_idx, sort_score=True)
            support_set.append((text, labels))

        with higher.innerloop_ctx(learner.pln, learner.inner_optimizer,
                                  copy_initial_weights=False, track_higher_grads=False) as (fpln, diffopt):
            
            INNER_tic = time.time()
            # Inner loop
            task_predictions, task_labels = [], []
            support_loss = []
            for text, labels in support_set:
                labels = torch.tensor(labels).to(device)
                input_dict = learner.rln.encode_text(text)
                _repr = learner.rln(input_dict)
                output = fpln(_repr)
                loss = learner.loss_fn(output, labels)
                diffopt.step(loss)
                pred = models.utils.make_prediction(output.detach())
                support_loss.append(loss.item())
                task_predictions.extend(pred.tolist())
                task_labels.extend(labels.tolist())
            INNER_toc = time.time() - INNER_tic
            all_adaptation_time.append(INNER_toc)

            acc, prec, rec, f1 = models.utils.calculate_metrics(task_predictions, task_labels)

            print('Support set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                        'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(support_loss), acc, prec, rec, f1))

            # Query set is now here!
            query_labels = torch.tensor(query_labels).to(device)
            query_input_dict = learner.rln.encode_text(query_text)
            with torch.no_grad():
                query_repr = learner.rln(query_input_dict)
                query_output = fpln(query_repr) # Output has size of torch.Size([16, 33]) [BATCH, CLASSES]
                query_loss = learner.loss_fn(query_output, query_labels)
            query_loss = query_loss.item()
            # print(output.detach().size())
            # output.detach().max(-1) max on each Batch, which will return [0] max, [1] indices
            query_output_softmax = F.softmax(query_output, -1)
            query_label_conf = query_output_softmax[np.arange(len(query_output_softmax)), query_labels] # Select labels in the softmax of 33 classes

            query_pred = models.utils.make_prediction(query_output.detach())
            query_acc, query_prec, query_rec, query_f1 = models.utils.calculate_metrics(query_pred.tolist(), query_labels.tolist())
            
            print('Query set metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, '
                'recall = {:.4f}, F1 score = {:.4f}'.format(np.mean(query_loss), query_acc, query_prec, query_rec, query_f1))

            all_losses.append(query_loss)
            all_predictions.extend(query_pred.tolist())
            all_labels.extend(query_labels.tolist())
            all_label_conf.extend(query_label_conf.tolist())

    acc, prec, rec, f1 = models.utils.calculate_metrics(all_predictions, all_labels)
    print('Test metrics: Loss = {:.4f}, accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
                'F1 score = {:.4f}'.format(np.mean(all_losses), acc, prec, rec, f1))
    return acc, prec, rec, f1, all_predictions, all_labels, all_label_conf, all_adaptation_time

In [11]:
tic = time.time()
print('----------Testing on test set starts here----------')

accuracies, precisions, recalls, f1s = [], [], [], []
all_adapt_time = []
# Data for Visualization: [data_idx, label, label_conf, pred]
data_for_visual = []

for test_dataset in test_datasets:
    print('Testing on {}'.format(test_dataset.__class__.__name__))
    test_dataloader = data.DataLoader(test_dataset, batch_size=mini_batch_size, shuffle=False,
                                      collate_fn=datasets.utils.batch_encode)
    acc, prec, rec, f1, all_pred, all_label, all_label_conf, all_adaptation_time = evaluate(dataloader=test_dataloader, updates=updates, 
                                                mini_batch_size=mini_batch_size, dataname=test_dataset.__class__.__name__)
    
    data_ids = [test_dataset.__class__.__name__ + str(i) for i in range(len(all_label))]
    data_for_visual.extend(list(zip(data_ids, all_label, all_label_conf, all_pred)))
    all_adapt_time.extend(all_adaptation_time)
#     print(data_ids)
#     print(all_label)
#     raise Exception("BREAKPOINT")
    
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1s.append(f1)


print()
print("COPY PASTA - not really but ok")
for row in accuracies:
    print(row)
print()
print('Overall test metrics: Accuracy = {:.4f}, precision = {:.4f}, recall = {:.4f}, '
            'F1 score = {:.4f}'.format(np.mean(accuracies), np.mean(precisions), np.mean(recalls), np.mean(f1s)))

toc = time.time() - tic
print(f"Total Time used: {toc//60} minutes")

----------Testing on test set starts here----------
Testing on YelpDataset
Query ID 0/475
Support set metrics: Loss = 0.4551, accuracy = 0.8750, precision = 0.8739, recall = 0.8767, F1 score = 0.8722
Query set metrics: Loss = 0.5332, accuracy = 0.8125, precision = 0.9000, recall = 0.8000, F1 score = 0.8018
Query ID 1/475
Support set metrics: Loss = 0.5381, accuracy = 0.8375, precision = 0.8394, recall = 0.8400, F1 score = 0.8360
Query set metrics: Loss = 0.7235, accuracy = 0.8125, precision = 0.8000, recall = 0.8333, F1 score = 0.7685
Query ID 2/475
Support set metrics: Loss = 0.4322, accuracy = 0.9000, precision = 0.9018, recall = 0.9000, F1 score = 0.8989
Query set metrics: Loss = 0.8163, accuracy = 0.7500, precision = 0.8476, recall = 0.7667, F1 score = 0.7597
Query ID 3/475
Support set metrics: Loss = 0.4383, accuracy = 0.8875, precision = 0.9050, recall = 0.9000, F1 score = 0.8913
Query set metrics: Loss = 0.9683, accuracy = 0.6250, precision = 0.6667, recall = 0.6167, F1 score = 

Support set metrics: Loss = 0.5007, accuracy = 0.8125, precision = 0.8123, recall = 0.8133, F1 score = 0.8088
Query set metrics: Loss = 0.4894, accuracy = 0.8750, precision = 0.9286, recall = 0.8750, F1 score = 0.8750
Query ID 36/475
Support set metrics: Loss = 0.5380, accuracy = 0.8500, precision = 0.8504, recall = 0.8533, F1 score = 0.8487
Query set metrics: Loss = 0.7982, accuracy = 0.8125, precision = 0.7714, recall = 0.7667, F1 score = 0.7313
Query ID 37/475
Support set metrics: Loss = 0.5061, accuracy = 0.8875, precision = 0.8919, recall = 0.8933, F1 score = 0.8886
Query set metrics: Loss = 0.9519, accuracy = 0.5625, precision = 0.5333, recall = 0.4500, F1 score = 0.4428
Query ID 38/475
Support set metrics: Loss = 0.5375, accuracy = 0.8000, precision = 0.8184, recall = 0.8133, F1 score = 0.8004
Query set metrics: Loss = 0.9534, accuracy = 0.6875, precision = 0.7933, recall = 0.7133, F1 score = 0.6921
Query ID 39/475
Support set metrics: Loss = 0.5767, accuracy = 0.8000, precision

Query set metrics: Loss = 1.2401, accuracy = 0.5000, precision = 0.6600, recall = 0.6267, F1 score = 0.5933
Query ID 71/475
Support set metrics: Loss = 0.5580, accuracy = 0.8000, precision = 0.8129, recall = 0.8100, F1 score = 0.8005
Query set metrics: Loss = 0.9337, accuracy = 0.5625, precision = 0.3800, recall = 0.4833, F1 score = 0.4254
Query ID 72/475
Support set metrics: Loss = 0.5086, accuracy = 0.8000, precision = 0.8159, recall = 0.8100, F1 score = 0.8001
Query set metrics: Loss = 1.1094, accuracy = 0.6250, precision = 0.6333, recall = 0.7300, F1 score = 0.6612
Query ID 73/475
Support set metrics: Loss = 0.5834, accuracy = 0.8000, precision = 0.8093, recall = 0.8067, F1 score = 0.8002
Query set metrics: Loss = 0.8214, accuracy = 0.6250, precision = 0.5667, recall = 0.5633, F1 score = 0.5548
Query ID 74/475
Support set metrics: Loss = 0.4867, accuracy = 0.8375, precision = 0.8356, recall = 0.8400, F1 score = 0.8345
Query set metrics: Loss = 1.1884, accuracy = 0.5625, precision =

Query set metrics: Loss = 0.6932, accuracy = 0.6250, precision = 0.6600, recall = 0.6167, F1 score = 0.5863
Query ID 106/475
Support set metrics: Loss = 0.5091, accuracy = 0.8500, precision = 0.8610, recall = 0.8600, F1 score = 0.8519
Query set metrics: Loss = 1.3524, accuracy = 0.3750, precision = 0.3833, recall = 0.4667, F1 score = 0.4038
Query ID 107/475
Support set metrics: Loss = 0.4357, accuracy = 0.9125, precision = 0.9179, recall = 0.9200, F1 score = 0.9142
Query set metrics: Loss = 1.6322, accuracy = 0.4375, precision = 0.4033, recall = 0.4167, F1 score = 0.3881
Query ID 108/475
Support set metrics: Loss = 0.5057, accuracy = 0.8750, precision = 0.8708, recall = 0.8733, F1 score = 0.8712
Query set metrics: Loss = 1.1859, accuracy = 0.3750, precision = 0.4524, recall = 0.3567, F1 score = 0.3200
Query ID 109/475
Support set metrics: Loss = 0.5343, accuracy = 0.8250, precision = 0.8270, recall = 0.8267, F1 score = 0.8210
Query set metrics: Loss = 1.0019, accuracy = 0.5000, precisi

Query set metrics: Loss = 0.6082, accuracy = 0.7500, precision = 0.7867, recall = 0.7600, F1 score = 0.7211
Query ID 141/475
Support set metrics: Loss = 0.5708, accuracy = 0.8125, precision = 0.8174, recall = 0.8167, F1 score = 0.8115
Query set metrics: Loss = 0.8229, accuracy = 0.5625, precision = 0.5000, recall = 0.5000, F1 score = 0.5000
Query ID 142/475
Support set metrics: Loss = 0.5014, accuracy = 0.8625, precision = 0.8768, recall = 0.8733, F1 score = 0.8647
Query set metrics: Loss = 0.5886, accuracy = 0.8750, precision = 0.8600, recall = 0.8500, F1 score = 0.8159
Query ID 143/475
Support set metrics: Loss = 0.4751, accuracy = 0.7875, precision = 0.7953, recall = 0.7933, F1 score = 0.7853
Query set metrics: Loss = 0.6185, accuracy = 0.8125, precision = 0.8333, recall = 0.8100, F1 score = 0.8169
Query ID 144/475
Support set metrics: Loss = 0.4528, accuracy = 0.8375, precision = 0.8455, recall = 0.8467, F1 score = 0.8387
Query set metrics: Loss = 0.9065, accuracy = 0.5625, precisi

...

Support set metrics: Loss = 0.5481, accuracy = 0.8500, precision = 0.8694, recall = 0.8400, F1 score = 0.8419
Query set metrics: Loss = 0.2714, accuracy = 0.9375, precision = 0.9375, recall = 0.9375, F1 score = 0.9167
Query ID 313/475
Support set metrics: Loss = 0.6907, accuracy = 0.8250, precision = 0.8856, recall = 0.8200, F1 score = 0.8323
Query set metrics: Loss = 0.5388, accuracy = 0.8125, precision = 0.8542, recall = 0.9083, F1 score = 0.8604
Query ID 314/475
Support set metrics: Loss = 0.6189, accuracy = 0.8125, precision = 0.8158, recall = 0.7900, F1 score = 0.7938
Query set metrics: Loss = 0.8925, accuracy = 0.8125, precision = 0.6786, recall = 0.6786, F1 score = 0.6735
Query ID 315/475
Support set metrics: Loss = 0.6002, accuracy = 0.8500, precision = 0.8941, recall = 0.8700, F1 score = 0.8735
Query set metrics: Loss = 0.8147, accuracy = 0.8125, precision = 0.8367, recall = 0.7143, F1 score = 0.7509
Query ID 316/475
Support set metrics: Loss = 0.5089, accuracy = 0.8625, preci

Support set metrics: Loss = 0.6670, accuracy = 0.7500, precision = 0.7914, recall = 0.7600, F1 score = 0.7428
Query set metrics: Loss = 0.5903, accuracy = 0.8750, precision = 0.8958, recall = 0.8750, F1 score = 0.8708
Query ID 348/475
Support set metrics: Loss = 0.6491, accuracy = 0.7875, precision = 0.8421, recall = 0.7800, F1 score = 0.7890
Query set metrics: Loss = 1.0123, accuracy = 0.6875, precision = 0.5833, recall = 0.6111, F1 score = 0.5630
Query ID 349/475
Support set metrics: Loss = 0.5900, accuracy = 0.8000, precision = 0.7861, recall = 0.8000, F1 score = 0.7771
Query set metrics: Loss = 0.8917, accuracy = 0.7500, precision = 0.5952, recall = 0.6429, F1 score = 0.6054
Query ID 350/475
Support set metrics: Loss = 0.5303, accuracy = 0.8625, precision = 0.8671, recall = 0.8700, F1 score = 0.8629
Query set metrics: Loss = 0.5959, accuracy = 0.8125, precision = 0.9375, recall = 0.8333, F1 score = 0.8500
Query ID 351/475
Support set metrics: Loss = 0.5841, accuracy = 0.8125, preci

Support set metrics: Loss = 0.6422, accuracy = 0.7500, precision = 0.7700, recall = 0.7300, F1 score = 0.7350
Query set metrics: Loss = 0.7698, accuracy = 0.8125, precision = 0.7619, recall = 0.7143, F1 score = 0.6905
Query ID 383/475
Support set metrics: Loss = 0.6127, accuracy = 0.8000, precision = 0.8347, recall = 0.7800, F1 score = 0.7898
Query set metrics: Loss = 0.8631, accuracy = 0.7500, precision = 0.7857, recall = 0.7333, F1 score = 0.7222
Query ID 384/475
Support set metrics: Loss = 0.5282, accuracy = 0.8500, precision = 0.8829, recall = 0.8600, F1 score = 0.8556
Query set metrics: Loss = 0.8984, accuracy = 0.6875, precision = 0.6167, recall = 0.6833, F1 score = 0.6233
Query ID 385/475
Support set metrics: Loss = 0.6228, accuracy = 0.7750, precision = 0.7895, recall = 0.7700, F1 score = 0.7615
Query set metrics: Loss = 1.5036, accuracy = 0.5625, precision = 0.7500, recall = 0.6146, F1 score = 0.6280
Query ID 386/475
Support set metrics: Loss = 0.4668, accuracy = 0.8750, preci

Support set metrics: Loss = 0.5795, accuracy = 0.8500, precision = 0.8871, recall = 0.8200, F1 score = 0.8327
Query set metrics: Loss = 0.6799, accuracy = 0.7500, precision = 0.7333, recall = 0.8000, F1 score = 0.7371
Query ID 418/475
Support set metrics: Loss = 0.5839, accuracy = 0.8500, precision = 0.8695, recall = 0.8500, F1 score = 0.8503
Query set metrics: Loss = 0.7500, accuracy = 0.8125, precision = 0.7500, recall = 0.7708, F1 score = 0.7250
Query ID 419/475
Support set metrics: Loss = 0.5724, accuracy = 0.8625, precision = 0.8811, recall = 0.8400, F1 score = 0.8478
Query set metrics: Loss = 0.9337, accuracy = 0.5000, precision = 0.7333, recall = 0.4583, F1 score = 0.5296
Query ID 420/475
Support set metrics: Loss = 0.7095, accuracy = 0.7250, precision = 0.7605, recall = 0.7000, F1 score = 0.7141
Query set metrics: Loss = 0.8986, accuracy = 0.6875, precision = 0.4630, recall = 0.6111, F1 score = 0.4963
Query ID 421/475
Support set metrics: Loss = 0.6092, accuracy = 0.8375, preci

Query ID 452/475
Support set metrics: Loss = 0.5680, accuracy = 0.8500, precision = 0.8815, recall = 0.8600, F1 score = 0.8587
Query set metrics: Loss = 0.3975, accuracy = 0.9375, precision = 1.0000, recall = 0.9524, F1 score = 0.9714
Query ID 453/475
Support set metrics: Loss = 0.5963, accuracy = 0.8250, precision = 0.8623, recall = 0.7900, F1 score = 0.8005
Query set metrics: Loss = 0.9078, accuracy = 0.6250, precision = 0.6667, recall = 0.7083, F1 score = 0.6542
Query ID 454/475
Support set metrics: Loss = 0.5933, accuracy = 0.8000, precision = 0.7974, recall = 0.8000, F1 score = 0.7754
Query set metrics: Loss = 0.5370, accuracy = 0.7500, precision = 0.6500, recall = 0.7000, F1 score = 0.6667
Query ID 455/475
Support set metrics: Loss = 0.5299, accuracy = 0.8875, precision = 0.9209, recall = 0.8700, F1 score = 0.8813
Query set metrics: Loss = 0.5229, accuracy = 0.9375, precision = 0.8611, recall = 0.8889, F1 score = 0.8730
Query ID 456/475
Support set metrics: Loss = 0.6147, accurac

In [12]:
_model_path0 = os.path.splitext(model_path)[0]
csv_filename = _model_path0 + "_update"+ str(updates) +"_results.csv" # for selective replay
with open(csv_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["data_idx", "label", "label_conf", "pred"])
    csv_writer.writerows(data_for_visual)
print(f"Done writing CSV File at {csv_filename}")

Done writing CSV File at /data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061_update5_results.csv


In [13]:
# Log Time for Inference
_model_path0 = os.path.splitext(model_path)[0]
time_txt_filename = _model_path0 + "_update"+ str(updates) +"_time_inference.csv" 
with open(time_txt_filename, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["time_id", "time"])
    csv_writer.writerow(["Total Time", f"{toc//60} minutes"])
    csv_writer.writerow(["mean Adapt Time", f"{np.mean(all_adapt_time)} s"])
print(f"Done writing Time CSV File at {time_txt_filename}")

Done writing Time CSV File at /data/model_runs/original_oml/aOML-order4-2023-01-08-srev-replay/OML-order4-id4-2023-01-08_21-01-31.303061_update5_time_inference.csv
